# Mapping and Plotting the ICESat

## Mapping

### Load the file

In [2]:
import os
from pathlib import Path
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/icesat/')
## list them up and check them
files= list(data_home.glob('*.H5'))

In [4]:
##### process it & subsetting
def glah14_to_df(filename,bounds):
    ## sp_ex = [103.643, 104.667, 12.375, 13.287]
    ## Bounds are [Longitude_West, Longitude_East, Latitude_South, Latitude_North]
    f = h5py.File(filename, 'r')
    lat = f['Data_40HZ']['Geolocation']['d_lat'][:]
    lon = f['Data_40HZ']['Geolocation']['d_lon'][:]
    elev = f['Data_40HZ']['Elevation_Surfaces']['d_elev'][:]
    sec = f['Data_40HZ']['Elevation_Corrections']['d_satElevCorr'][:]
    scf = f['Data_40HZ']['Quality']['sat_corr_flg'][:]
    satndx = f['Data_40HZ']['Quality']['i_satNdx'][:]
    dem = f['Data_40HZ']['Geophysical']['d_DEM_elv'][:]
    
    glah14_df = pd.DataFrame({'Latitude':lat,'Longitude':lon,'Elevation':elev,
                            's_El_Corr':sec, 's_Corr_f':scf,'in_sat':satndx,
                            'DEM':dem})
    #### Subsetting
    glah14_df_subset = glah14_df.loc[(glah14_df['Longitude']>=bounds[0]) 
                          & (glah14_df['Longitude']<=bounds[1])
                          & (glah14_df['Latitude']>=bounds[2])
                          & (glah14_df['Latitude']<=bounds[3])]
    return glah14_df_subset

In [5]:
### Spatial Bounds: 
tsl_sp_ex = [103.643, 104.667, 12.375, 13.287]
### test track of ICESat
test_df = glah14_to_df(files[1],tsl_sp_ex)

In [8]:
print(test_df)

         Latitude   Longitude  Elevation  s_El_Corr  s_Corr_f  in_sat    DEM
159517  13.286178  104.542759     59.483      0.000         0       0  60.30
159518  13.284618  104.542540     56.767      0.000         0       0  60.43
159519  13.283058  104.542322     58.253      0.000         0       0  60.55
159520  13.281498  104.542106     54.924      0.000         0       0  60.65
159521  13.279939  104.541892     48.373      0.000         0       0  60.75
...           ...         ...        ...        ...       ...     ...    ...
160097  12.382718  104.416207      7.871      0.173         2       3  11.87
160098  12.381162  104.415988      8.851      0.115         2       2  12.18
160099  12.379608  104.415769      7.519      0.000         0       0  12.51
160100  12.378055  104.415549      6.656      0.996         2      20  12.67
160101  12.376502  104.415327      9.499      0.000         0       0  12.76

[585 rows x 7 columns]


### Mapping

In [7]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(tsl_sp_ex, crs=ccrs.PlateCarree())
plt.scatter(test_df['Longitude'], test_df['Latitude'],s=1)

request = cimgt.Stamen('terrain-background')
ax.add_image(request, 9)
plt.title("Tracks on TSL")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Tracks on TSL')

### Plotting

In [6]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(test_df['Latitude'],test_df['Elevation'],'.',markersize=0.25, label='all heights')
h_leg=ax.legend()
plt.title('Heights from ICESat')
ax.set_xlabel('Latitude')
ax.set_ylabel('Elevation, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mapping the track with elevation information

In [11]:
fig = plt.figure(figsize=(5,5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(tsl_sp_ex, crs=ccrs.PlateCarree())
plt.scatter(test_df['Longitude'], test_df['Latitude'], s=2, c=test_df['Elevation'], alpha=.7, transform=ccrs.PlateCarree(), cmap='terrain')
plt.colorbar(fraction=0.0320, pad=0.02, label='Elevation (m)')

##load Google Sat.Map
#request = cimgt.GoogleTiles(style='satellite')
request = cimgt.Stamen('terrain-background')
ax.add_image(request, 9)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Removing outliers using flag information in itself